<a href="https://colab.research.google.com/github/NguyenPhuc213/Ynus213/blob/main/Detect_acupuncture_points_FinalAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#GetImageFromVideo 
#Code này chạy trên VS Code sau đó load data lên ColabColab
import cv2
import time

def main():
    cap = cv2.VideoCapture('D:\_PHUC\_SV19\HK II 2021 - 2022\Thi giac may\Video source\BanPhuc.mp4')
    # Resolution 640*480
    time.sleep(1)
    if cap is None or not cap.isOpened():
        print('Khong the mo file video')
        return
    cv2.namedWindow('Image', cv2.WINDOW_AUTOSIZE);
    n = 1
    dem = 0
    while True:
        [success, img] = cap.read()
        ch = cv2.waitKey(30)
        if success:       
            imgROI = img[180:(480+680),50:(300+480)] # Tao ra anh 680x480

            imgROI = cv2.resize(imgROI,(250,250))
            cv2.imshow('Image', imgROI)
        else:
            break
        if n%4 == 0:
            filename = 'D:\_PHUC\_SV19\HK II 2021 - 2022\Thi giac may\Image_Video\BanPhuc\BanPhuc_%04d.bmp'%(dem)
            cv2.imwrite(filename,imgROI)
            dem = dem + 1
        n = n + 1
    return

if __name__ == "__main__":
    main()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import cv2
from os import listdir
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
def plot_history(history_fine):
  f1 = history_fine.history['acc']
  val_f1 = history_fine.history['val_acc']
  loss = history_fine.history['loss']
  val_loss = history_fine.history['val_loss']
  plt.figure(figsize=(8, 8))
  plt.subplot(2, 1, 1)
  plt.plot(f1, label='Acc')
  plt.plot(val_f1, label='Validation Acc')
  plt.legend(loc='lower right')
  plt.title('Accuracy')
  plt.subplot(2, 1, 2)
  plt.plot(loss, label='Loss')
  plt.plot(val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Loss')
  plt.xlabel('epoch')
  plt.show()

In [ ]:
def create_img_dataset(raw_folder,dataset_name):

    target_size = (680, 480)
    pictures = []
    labels = []
    for folder in listdir(raw_folder):
      for file in listdir(raw_folder + "/" + folder)[:1000]:
        pictures.append(cv2.resize(cv2.imread(raw_folder + "/" + folder + "/" + file,cv2.IMREAD_GRAYSCALE)
                                              ,dsize = target_size))
        labels.append(folder)

    pictures = np.array(pictures)
    labels = np.array(labels)

    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)

    with open (dataset_name, mode = 'wb') as file:
    # dump information to that file
      pickle.dump((pictures,labels), file)

    file.close()

def load_data(dataset_path):

    file = open(dataset_path, mode = 'rb')
    (pictures, labels) = pickle.load (file)
    file.close()

    return pictures, labels

In [ ]:
raw_folder = "/content/drive/MyDrive/BanPhuc"

create_img_dataset(raw_folder= raw_folder,dataset_name= 'data.pickle')
dataset_path = "data.pickle"

In [ ]:
X_DATA_PATH = 'data.pickle'
Y_DATA_PATH = 'train.csv'

In [ ]:
import pandas as pd
y_data = pd.read_csv(Y_DATA_PATH)
y_data.head(None)

In [ ]:
import glob
import pickle
import numpy as np
x_data = pickle.load(open(X_DATA_PATH, 'rb'))
x_data = np.array(x_data, dtype = 'float')
x_data /= 255
print('Shape of x data: ', x_data.shape)

In [ ]:
import matplotlib.pylab as plt
import pandas as pd
import random
import cv2

n = x_data.shape[0]
plt.figure(figsize = (15, 15))
plt.subplots_adjust(hspace = .2)
for i in range(6):
    plt.subplot(3, 2, i + 1)
    k = random.randint(0, 299)
    img = x_data[k]
    points = list(y_data.iloc[k])
    points = [tuple(points[i : i + 2]) for i in range(0, len(points), 2)]
    for x, y in points:
        cv2.circle(img, (int(x), int(y)), 2, (0, 0, 0), 2)
    plt.imshow(img)

_ = plt.suptitle('Sample images', size = 20)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
#x_data = x_data.reshape(-1, (150,150), 1)
input_shape = x_data.shape[1:4]
y_data = np.array(y_data, dtype = 'float')
num_class = y_data.shape[1]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.1)
print('Input shape: ', input_shape)
print('Number of output: ', num_class)
print('x train shape: ', x_train.shape)
print('y train shape: ', y_train.shape)
print('x test shape: ', x_test.shape)
print('y test shape: ', y_test.shape)

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, MaxPooling2D, Flatten, LeakyReLU, Convolution2D, MaxPool2D
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape = input_shape))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
# Dense
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
# model.add(Dense(num_class, activation='softmax'))
model.add(Dense(num_class))
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
model.compile(
    optimizer = 'Adam',
    loss = "mean_squared_error",
    metrics = ['mae']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_mae', patience = 25)
hist = model.fit(x_train, y_train, epochs = 100, batch_size = 16, validation_split = 0.1).history

In [ ]:
final_loss, final_accuracy = model.evaluate(x_test, y_test)
print('Final loss: {:.2f}'.format(final_loss))
print('Final accuracy: {:.2f}'.format(final_accuracy))

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
model_history = pd.DataFrame(hist)
# add epoch column
model_history['epoch'] = np.arange(1, len(model_history) + 1)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (8, 10))
epochs = model_history.shape[0]
ax1.plot(np.arange(0, epochs), model_history['mae'], label = 'Training accuracy')
ax1.plot(np.arange(0, epochs), model_history['val_mae'], label = 'Validation accuracy')
ax1.legend(loc = 'lower right')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax2.plot(np.arange(0, epochs), model_history['loss'], label = 'Training loss')
ax2.plot(np.arange(0, epochs), model_history['val_loss'], label = 'Validation loss')
ax2.legend(loc = 'upper right')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
plt.tight_layout()
plt.show()

In [ ]:
PATH = '/content/drive/MyDrive/AI/'
MODEL_NAME = "Model_MAE" + str(round(final_accuracy)) + ".h5"
model.save(PATH + MODEL_NAME)

In [ ]:
from keras.models import load_model
model = load_model(PATH + MODEL_NAME)

In [ ]:
import matplotlib.pylab as plt
import cv2
pred = model.predict(x_test)
n = x_test.shape[0]
plt.figure(figsize = (15, 15))
plt.subplots_adjust(hspace = .2)
plt.subplot(3, 2, i + 1)
#k = random.randint(0, 10)
k= 0
img = x_test[k].reshape(150,150,3)
points = pred[k,:]
#points = [tuple(points[i : i + 2]) for i in range(0, len(points), 2)]
print(points)
cv2.circle(img, (int(x), int(y)), 2, (0, 0, 0), 2)
plt.imshow(img)  
_ = plt.suptitle('Sample images', size = 20)